<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
import numpy as np
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [3]:
response.status_code

200

We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [4]:
parameters = {'lat':-33.87, 'lon':151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [5]:
response_pass = requests.get("http://api.open-notify.org/iss-pass.json", params = parameters)
#passing params in get requests > turns it into URL parameters ('query strings') > specify/sort content on page

Print the response header:

In [6]:
response_pass.headers
#what does this do? what's the significance of this result?

{'Server': 'nginx/1.10.3', 'Date': 'Tue, 22 Jun 2021 02:34:59 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}

Print the content of the response (the data that the server returned):

In [7]:
response_pass.content

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1624329299, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 311, \n      "risetime": 1624338841\n    }, \n    {\n      "duration": 655, \n      "risetime": 1624344436\n    }, \n    {\n      "duration": 591, \n      "risetime": 1624350291\n    }, \n    {\n      "duration": 395, \n      "risetime": 1624356274\n    }, \n    {\n      "duration": 391, \n      "risetime": 1624362191\n    }\n  ]\n}\n'

Note that this is a Python byte string:

In [8]:
print(type(response_pass.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [9]:
response_pass.headers['content-type']
#shows that response_pass is in json format
#Requests library comes with one built-in JSON parser > use requests.get('url').json() to parse it as a JSON object
#(con't) then the value for each key of the response results can be parsed easily like below:
import requests

#r = requests.get('http://httpbin.org/get')
#response = r.json()
#print(r.json())
#print(response['args'])
#print(response['headers'])
#print(response['headers']['Accept'])
#print(response['headers']['Accept-Encoding'])
#print(response['origin'])
#print(response['url'])

JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [10]:
response_pass.json()

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1624329299,
  'latitude': -33.87,
  'longitude': 151.21,
  'passes': 5},
 'response': [{'duration': 311, 'risetime': 1624338841},
  {'duration': 655, 'risetime': 1624344436},
  {'duration': 591, 'risetime': 1624350291},
  {'duration': 395, 'risetime': 1624356274},
  {'duration': 391, 'risetime': 1624362191}]}

What kind of object did this give us?

In [11]:
print(type(response_pass.json()))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [12]:
pass_times = response_pass.json()['response']
print(type(pass_times))
print(pass_times)

<class 'list'>
[{'duration': 311, 'risetime': 1624338841}, {'duration': 655, 'risetime': 1624344436}, {'duration': 591, 'risetime': 1624350291}, {'duration': 395, 'risetime': 1624356274}, {'duration': 391, 'risetime': 1624362191}]


Now extract the `risetime` strings into an array called `srisetimes`:

In [13]:
list_srisetimes = []

for d in pass_times:
    time = d['risetime']
    list_srisetimes.append(time)

srisetimes = np.array(list_srisetimes)
print(srisetimes)

[1624338841 1624344436 1624350291 1624356274 1624362191]


```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [14]:
risetimes = []

for rt in srisetimes:
    time = datetime.fromtimestamp(rt)
    risetimes.append(time)
    print(risetimes)


[datetime.datetime(2021, 6, 22, 13, 14, 1)]
[datetime.datetime(2021, 6, 22, 13, 14, 1), datetime.datetime(2021, 6, 22, 14, 47, 16)]
[datetime.datetime(2021, 6, 22, 13, 14, 1), datetime.datetime(2021, 6, 22, 14, 47, 16), datetime.datetime(2021, 6, 22, 16, 24, 51)]
[datetime.datetime(2021, 6, 22, 13, 14, 1), datetime.datetime(2021, 6, 22, 14, 47, 16), datetime.datetime(2021, 6, 22, 16, 24, 51), datetime.datetime(2021, 6, 22, 18, 4, 34)]
[datetime.datetime(2021, 6, 22, 13, 14, 1), datetime.datetime(2021, 6, 22, 14, 47, 16), datetime.datetime(2021, 6, 22, 16, 24, 51), datetime.datetime(2021, 6, 22, 18, 4, 34), datetime.datetime(2021, 6, 22, 19, 43, 11)]


Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [15]:
for t in risetimes:
    time = t.strftime('%d/%m/%y %I:%M')
    print(time)

22/06/21 01:14
22/06/21 02:47
22/06/21 04:24
22/06/21 06:04
22/06/21 07:43


Here is an endpoint that tells us who is onboard:

In [22]:
response_astros = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [17]:
response_astros.content

b'{"people": [{"name": "Mark Vande Hei", "craft": "ISS"}, {"name": "Oleg Novitskiy", "craft": "ISS"}, {"name": "Pyotr Dubrov", "craft": "ISS"}, {"name": "Thomas Pesquet", "craft": "ISS"}, {"name": "Megan McArthur", "craft": "ISS"}, {"name": "Shane Kimbrough", "craft": "ISS"}, {"name": "Akihiko Hoshide", "craft": "ISS"}, {"name": "Nie Haisheng", "craft": "Tiangong"}, {"name": "Liu Boming", "craft": "Tiangong"}, {"name": "Tang Hongbo", "craft": "Tiangong"}], "number": 10, "message": "success"}'

In [31]:
number_of_astros = response_astros.json()["number"]
print(number_of_astros)

names_crafts = response_astros.json()["people"]
print(names_crafts)


##############
list_names = []
for n in names_crafts:
    name = n['name']
    list_names.append(name)

names = np.array(list_names)
print(names)

10
[{'name': 'Mark Vande Hei', 'craft': 'ISS'}, {'name': 'Oleg Novitskiy', 'craft': 'ISS'}, {'name': 'Pyotr Dubrov', 'craft': 'ISS'}, {'name': 'Thomas Pesquet', 'craft': 'ISS'}, {'name': 'Megan McArthur', 'craft': 'ISS'}, {'name': 'Shane Kimbrough', 'craft': 'ISS'}, {'name': 'Akihiko Hoshide', 'craft': 'ISS'}, {'name': 'Nie Haisheng', 'craft': 'Tiangong'}, {'name': 'Liu Boming', 'craft': 'Tiangong'}, {'name': 'Tang Hongbo', 'craft': 'Tiangong'}]
['Mark Vande Hei' 'Oleg Novitskiy' 'Pyotr Dubrov' 'Thomas Pesquet'
 'Megan McArthur' 'Shane Kimbrough' 'Akihiko Hoshide' 'Nie Haisheng'
 'Liu Boming' 'Tang Hongbo']


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [34]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code
    if response.status_code == 200:
        return(False, response.status_code)
    else:
        return(True, response.status_code)

2. Test your response handler on some correct and incorrect API calls.

In [40]:
response_astros = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response_astros)[0] == True:
    print('API call failed. Resolve issue before continuing!')
else:
    print('API call success!')
    
response_pass = requests.get("http://api.open-notify.org/iss-pass.json") #will be status: fail because requires lat and lon input
if handleResponse(response_pass)[0] == True:
    print('API call failed. Resolve issue before continuing!')
else:
    print('API call success!')

API call success!
API call failed. Resolve issue before continuing!


>

>

>



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



